# 23 Day 05

https://adventofcode.com/2023/day/5


In [2]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)

def test(method, input, expected):
    actual = method(*input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')


Let's make sure that the example contains all the actual map value

In [18]:
puzzle.input_data

import re
re.findall(r'\w+-to-\w+', puzzle.input_data)

['seed-to-soil',
 'soil-to-fertilizer',
 'fertilizer-to-water',
 'water-to-light',
 'light-to-temperature',
 'temperature-to-humidity',
 'humidity-to-location']

Ok, it matches the example. Whew!

In [220]:
from collections import defaultdict


class Almanac:

    def __init__(self, input_data:str):
        input_blocks = input_data.split('\n\n')
        
        self.seeds = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        
        maps = {}
        # Do I need this?
        # maps = {('seed', 'seed'): seeds}

        # I could not get this to go in one block. I'll ask for some help here.
        # (\w+)-to-(\w+) map:\n(:?(\d+) (\d+) (\d+)\n?)+
        # https://regex101.com/r/EtIaGl/1
        # Deletion Link: https://regex101.com/delete/pRW1eIlXymmnzhMO4O44tLemCOriUzaGGgkA
        for block in input_blocks[1:]:
            # print(block)
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()

            mapping = {}
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                dest_start, source_start, range_len = m
                for src, dest in zip(range(int(source_start), int(source_start) + int(range_len)), range(int(dest_start) , int(dest_start) + int(range_len))):
                    # TODO Validate this isn't already populated in case there are a list of options
                    mapping[src] = dest
            maps[map_label] = mapping
            
        self.maps = maps
            # TODO: There must be a better way to accomplish this in Pandas...
            # for indx, value in df[map_label[0]].items():
            #     # print(df[indx])
            #     print(df.iloc[indx])
            # row = df.loc[df[map_label[1]] == some_value]
            # df[map_label[1]] = df.apply(lambda row: mapping[row[map_label[0]]] if row[map_label[0]] in mapping else row[map_label[0]], axis=1)

    def translate(self, source:str, id:int, dest: str):
        # I could switch to a graph with nodes if this was ever more robust
        if (source, dest) in self.maps.keys():
            return self.maps[(source, dest)][id] if id in self.maps[(source, dest)] else id
        else:
            # print(f"Couldn't find {source} -> {dest} directly.")
            # This assumes there is only one path. No need to build graph
            _, next_dest = next(m for m in self.maps.keys() if m[0] == source)
            # print(f'Next hop {next_dest} -> {dest}')
            next_id = id if id not in self.maps[(source, next_dest)] else self.maps[(source, next_dest)][id]
            return self.translate(next_dest, next_id, dest)


In [219]:
example = puzzle.examples[0]

# Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
# Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
# Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
# Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
print('Part A Test Cases:')
a = Almanac(example.input_data)
for (input, expected) in [(('seed', 79, 'soil'), 81), (('seed', 14, 'soil'), 14), (('seed', 55, 'soil'), 57), 
                          (('seed', 13, 'soil'), 13), (('seed', 79, 'fertilizer'), 81), (('seed', 79, 'water'), 81),
                          (('seed', 79, 'light'), 74), (('seed', 79, 'temperature'), 78), (('seed', 79, 'humidity'), 78),
                          (('seed', 79, 'location'), 82), (('seed', 14, 'location'), 43), (('seed', 55, 'location'), 86),
                          (('seed', 13, 'location'), 35)]:
    print(*input)
    test(a.translate, input, expected)

Part A Test Cases:
seed 79 soil
	☑ - translate(('seed', 79, 'soil')) = 81 = 81
seed 14 soil
	☑ - translate(('seed', 14, 'soil')) = 14 = 14
seed 55 soil
	☑ - translate(('seed', 55, 'soil')) = 57 = 57
seed 13 soil
	☑ - translate(('seed', 13, 'soil')) = 13 = 13
seed 79 fertilizer
	☑ - translate(('seed', 79, 'fertilizer')) = 81 = 81
seed 79 water
	☑ - translate(('seed', 79, 'water')) = 81 = 81
seed 79 light
	☑ - translate(('seed', 79, 'light')) = 74 = 74
seed 79 temperature
	☑ - translate(('seed', 79, 'temperature')) = 78 = 78
seed 79 humidity
	☑ - translate(('seed', 79, 'humidity')) = 78 = 78
seed 79 location
	☑ - translate(('seed', 79, 'location')) = 82 = 82
seed 14 location
	☑ - translate(('seed', 14, 'location')) = 43 = 43
seed 55 location
	☑ - translate(('seed', 55, 'location')) = 86 = 86
seed 13 location
	☑ - translate(('seed', 13, 'location')) = 35 = 35


In [217]:
a = Almanac(example.input_data)
print(f'Solution A: {min([a.translate('seed', s, 'location') for s in a.seeds])}')

Solution A: 35


In [218]:
a = Almanac(puzzle.input_data)
print(f'Solution A: {min([a.translate('seed', s, 'location') for s in a.seeds])}')

MemoryError: 

Bummer! Memory error. I could prune down and eliminate anything that didn't have a seed, but I'm afraid that part B will look for a more general solution. Let's try to keep as little in memory as possible. I try a range or generator where needed first.

In [39]:
import re
from collections import defaultdict


class LowMemoryAlmanac:

    def __init__(self, input_data:str):
        input_blocks = input_data.split('\n\n')
        
        self.seeds = [int(s) for s in re.findall(r'\d+', input_blocks[0])]
        
        maps = {}

        for block in input_blocks[1:]:
            map_label = re.match(r'(\w+)-to-(\w+) map:', block).groups()
            # print(f'Working on {map_label[0]} => {map_label[1]}')
            
            ranges = []
            for m in re.findall(r'(?:(\d+) (\d+) (\d+))+?', block):
                # TODO don't include anything that's not linked in an earlier map
                dest_start, source_start, range_len = m
                source_range = range(int(source_start), int(source_start) + int(range_len))
                dest_range = range(int(dest_start) , int(dest_start) + int(range_len))
                ranges.append((source_range, dest_range))
            maps[map_label] = ranges
            
        self.maps = maps
        
    def translate(self, source:str, id:int, dest: str):
        # I could switch to a graph with nodes if this was ever more robust
        if (source, dest) in self.maps.keys():
            # Direct translation
            ranges = next(((source_range, dest_range) for source_range, dest_range in self.maps[(source, dest)] if id in source_range), None)
            if not ranges:
                return id
            source_range, dest_range = ranges
            return next(dest for src, dest in zip(source_range, dest_range) if src == id)
        else:
            # print(f"Couldn't find {source} -> {dest} directly.")
            # This assumes there is only one path. No need to build graph
            _, next_dest = next(m for m in self.maps.keys() if m[0] == source)
            # print(f'Next hop {next_dest} -> {dest}')
            next_ranges = next(((source_range, next_range) for source_range, next_range in self.maps[(source, next_dest)] if id in source_range), None)
            next_id = id
            if next_ranges:
                source_range, dest_range = next_ranges
                next_id = next(dest for src, dest in zip(source_range, dest_range) if src == id)
            return self.translate(next_dest, next_id, dest)


In [40]:
a = LowMemoryAlmanac(puzzle.examples[0].input_data)

# Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
# Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
# Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
# Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
print('Low Memory Part A Test Cases:')
for (input, expected) in [(('seed', 79, 'soil'), 81), (('seed', 14, 'soil'), 14), (('seed', 55, 'soil'), 57), 
                          (('seed', 13, 'soil'), 13), (('seed', 79, 'fertilizer'), 81), (('seed', 79, 'water'), 81),
                          (('seed', 79, 'light'), 74), (('seed', 79, 'temperature'), 78), (('seed', 79, 'humidity'), 78),
                          (('seed', 79, 'location'), 82), (('seed', 14, 'location'), 43), (('seed', 55, 'location'), 86),
                          (('seed', 13, 'location'), 35)]:
    print(*input)
    test(a.translate, input, expected)


Low Memory Part A Test Cases:
seed 79 soil
	☑ - translate(('seed', 79, 'soil')) = 81 = 81
seed 14 soil
	☑ - translate(('seed', 14, 'soil')) = 14 = 14
seed 55 soil
	☑ - translate(('seed', 55, 'soil')) = 57 = 57
seed 13 soil
	☑ - translate(('seed', 13, 'soil')) = 13 = 13
seed 79 fertilizer
	☑ - translate(('seed', 79, 'fertilizer')) = 81 = 81
seed 79 water
	☑ - translate(('seed', 79, 'water')) = 81 = 81
seed 79 light
	☑ - translate(('seed', 79, 'light')) = 74 = 74
seed 79 temperature
	☑ - translate(('seed', 79, 'temperature')) = 78 = 78
seed 79 humidity
	☑ - translate(('seed', 79, 'humidity')) = 78 = 78
seed 79 location
	☑ - translate(('seed', 79, 'location')) = 82 = 82
seed 14 location
	☑ - translate(('seed', 14, 'location')) = 43 = 43
seed 55 location
	☑ - translate(('seed', 55, 'location')) = 86 = 86
seed 13 location
	☑ - translate(('seed', 13, 'location')) = 35 = 35


In [42]:
a = LowMemoryAlmanac(puzzle.input_data)

print(f'Solution A: {min([a.translate('seed', s, 'location') for s in a.seeds])}')

Solution A: 88151870


Wow! 31m 52.9s to finish. That's not ideal, but better than out of memory. I wonder if I could work backward from the lowest location to valid seed to be faster.